In [48]:
LABEL_MIN_COUNT = 50

In [1]:
%load_ext lab_black

In [93]:
import itertools

import pandas as pd

import spacy
from spacy import displacy

import gensim
import fasttext
import umap
import altair as alt

from tqdm.notebook import tqdm

## Load Data and NLP Model

In [10]:
df_recipes = pd.read_csv("../../data/sparkrecipes_base.csv").sample(100000)
df_recipes

recipe_id                                              title  \
274504    1176156                            Shrimp Pesto Fettuccine   
384873    1797779  Roasted Brussels Sprouts with Maple-Balsamic G...   
316009    1404234                            Honey Banana Fruit Cake   
199062     772713                       Whole Wheat homemade lasagna   
148579     560800                                  Runza Bread Dough   
...           ...                                                ...   
442912    2131186                     Banana Chocolate Chip Pancakes   
397286    1867301                                   Crockpot Oatmeal   
208608     821665    Squash and Chicken Stir Fry - 1 cup per serving   
322549    1438479              Trident Bookseller Vegan Cashew Chili   
566528    2954703                     Oven Baked Macaroni and Cheese   

        total_calories                                                url  \
274504           299.0  https://recipes.sparkpeople.com/recipe-detail....   
384873            82.4  https://recipes.sparkpeople.com/recipe-detail....   
316009           271.7  https://recipes.sparkpeople.com/recipe-detail....   
199062           249.4  https://recipes.sparkpeople.com/recipe-detail....   
148579           181.4  https://recipes.sparkpeople.com/recipe-detail....   
...                ...                                                ...   
442912           181.9  https://recipes.sparkpeople.com/recipe-detail....   
397286           159.9  https://recipes.sparkpeople.com/recipe-detail....   
208608           104.0  https://recipes.sparkpeople.com/recipe-detail....   
322549           336.6  https://recipes.sparkpeople.com/recipe-detail....   
566528           336.7  https://recipes.sparkpeople.com/recipe-detail....   

        servings  
274504       5.0  
384873       4.0  
316009      24.0  
199062      12.0  
148579      24.0  
...          ...  
442912       7.0  
397286       3.0  
208608      16.0  
322549       8.0  
566528       8.0  

[100000 rows x 5 columns]

## Label Extraction

In [11]:
nlp = spacy.load("en_core_web_lg")

In [12]:
sents = df_recipes.title.map(lambda t: gensim.utils.simple_preprocess(t, deacc=True))

In [13]:
bigram = gensim.models.Phrases(sents, min_count=150, threshold=1, scoring="default")
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [14]:
def make_bigrams(sents, bigram_mod):
    return [bigram_mod[s] for s in sents]


def compute_labels(bigrams):
    result = []
    for i, b in tqdm(list(enumerate(bigrams))):
        doc = nlp(" ".join(b))
        labels = set([chunk.root.text for chunk in doc.noun_chunks])
        if len(labels) == 0 and len(b) > 0:
            labels = set([b[0]])
        result.append(labels)
        if i % 1000 == 0:
            print(f"Iteration {i}: {b} -> {labels}")
    return result

In [15]:
bigrams = make_bigrams(sents, bigram_mod)

df_recipes["labels"] = compute_labels(bigrams)

Iteration 0: ['shrimp', 'pesto', 'fettuccine'] -> {'shrimp'}
Iteration 1000: ['mama', 'love', 'lighter', 'cinnamon', 'rolls'] -> {'mama', 'rolls'}
Iteration 2000: ['tuna', 'with', 'twist'] -> {'tuna', 'twist'}
Iteration 3000: ['broccoli', 'salad', 'no', 'mayo'] -> {'salad'}
Iteration 4000: ['slow_cooker', 'pork', 'giambotta'] -> {'giambotta'}
Iteration 5000: ['terri', 'salsa'] -> {'salsa'}
Iteration 6000: ['banana', 'chocolate_chip', 'muffins'] -> {'muffins'}
Iteration 7000: ['chocolate_chip', 'cookies', 'with', 'chick', 'peas', 'and', 'stevia'] -> {'peas', 'stevia', 'cookies'}
Iteration 8000: ['sweet', 'southern', 'reduction'] -> {'reduction'}
Iteration 9000: ['dark', 'chocolate', 'peanut_butter', 'banana', 'wrap'] -> {'wrap'}
Iteration 10000: ['orzo', 'and', 'chicken'] -> {'orzo', 'chicken'}
Iteration 11000: ['turkey', 'and_rice'] -> {'and_rice'}
Iteration 12000: ['celery', 'and', 'apple', 'salad'] -> {'salad'}
Iteration 13000: ['brown', 'butter', 'and', 'oat', 'scone', 'recipe'] -> 

In [16]:
df_recipes.sample(10)

recipe_id                                              title  \
147380     556357                          Fat Elvis (Protein Shake)   
165271     626333                                        Easy Hummus   
63685      232468                                     Cake in a bowl   
8283        32377                                    stuffed burgers   
419959    1993650                         Mushroom Chicken Casserole   
487758    2413120                                               GORP   
547181    2812622                            Chilly Day Chicken Soup   
85310      316979                                   Blueberry Buckle   
147302     556029                       Lemon Garlic Chicken w/Honey   
137991     519627  Blueberry muffins (Taste of Home Quick Cooking...   

        total_calories                                                url  \
147380           558.3  https://recipes.sparkpeople.com/recipe-detail....   
165271           156.8  https://recipes.sparkpeople.com/recipe-detail....   
63685            215.8  https://recipes.sparkpeople.com/recipe-detail....   
8283             214.6  https://recipes.sparkpeople.com/recipe-detail....   
419959           393.7  https://recipes.sparkpeople.com/recipe-detail....   
487758           368.8  https://recipes.sparkpeople.com/recipe-detail....   
547181           187.5  https://recipes.sparkpeople.com/recipe-detail....   
85310            409.4  https://recipes.sparkpeople.com/recipe-detail....   
147302           169.8  https://recipes.sparkpeople.com/recipe-detail....   
137991           267.2  https://recipes.sparkpeople.com/recipe-detail....   

        servings             labels  
147380       1.0    {protein_shake}  
165271       5.0           {hummus}  
63685       20.0       {bowl, cake}  
8283         1.0          {burgers}  
419959       6.0        {casserole}  
487758      10.0             {gorp}  
547181       2.0             {soup}  
85310        9.0           {buckle}  
147302       6.0            {honey}  
137991      12.0  {taste, magazine}

In [17]:
def get_all_labels(df):
    return pd.Series(itertools.chain(*df["labels"]))

In [18]:
all_labels = get_all_labels(df_recipes)

In [20]:
all_labels.value_counts().nlargest(100).plot.bar(figsize=(20, 15))

In [33]:
label_val_counts = all_labels.value_counts()
label_val_counts.shape

(7677,)

In [34]:
label_val_counts.hist(bins=200, log=True, figsize=(10, 7))

In [51]:
relevant_labels = set(
    label_val_counts[label_val_counts >= LABEL_MIN_COUNT].index.tolist()
)
len(relevant_labels)

357

## Evaluation

In [60]:
df_recipes["filtered_labels"] = df_recipes.labels.map(
    lambda labels: labels & relevant_labels
)

In [61]:
label_avail_mask = df_recipes.filtered_labels.map(lambda l: len(l) > 0)

In [90]:
df_recipes[label_avail_mask].sample(20)

recipe_id                                              title  \
241523     995818                                grain free meatloaf   
260712    1100034                                  Tomato Basil Soup   
444710    2141871  Grilled Lamb Chops w/Roasted Potatoes & Green ...   
419318    1989916  Low Carb Chocolate Peppermint or Peanut Butter...   
448084    2162103                                Turkey Sheppard Pie   
31623      118429                    Southwest Bean Chili - Homemade   
20741       80848  Stuffed Peppers with rice, lentils, roasted to...   
185904     708258                           Potato Soup with carrots   
313097    1389384                                        Faked Beans   
290432    1265988                           Root Vegetable Casserole   
480964    2367804                         Meatball Madness Meatballs   
387524    1813608         Corrected Kathy's Zucchini Pineapple Bread   
269951    1151618    Pinto Beans With Ground Turkey & Rotel Tomatoes   
155771     588507                        Sweet and Spicy Asian Wings   
551593    2845944                            Easy Creamy Caulie Soup   
344399    1567289                       Drew's Healthy Vegan Cookies   
345416    1573062                spinach & mushroom egg white omelet   
567723    2968696      Carol's No Bake Peanut Butter Oatmeal Cookies   
92248      341765                   Jacket potatoe with cheesy beans   
102022     378303                                 spinach side/sauce   

        total_calories                                                url  \
241523           321.7  https://recipes.sparkpeople.com/recipe-detail....   
260712            71.5  https://recipes.sparkpeople.com/recipe-detail....   
444710           489.2  https://recipes.sparkpeople.com/recipe-detail....   
419318            89.7  https://recipes.sparkpeople.com/recipe-detail....   
448084           280.8  https://recipes.sparkpeople.com/recipe-detail....   
31623            216.6  https://recipes.sparkpeople.com/recipe-detail....   
20741            301.4  https://recipes.sparkpeople.com/recipe-detail....   
185904           116.9  https://recipes.sparkpeople.com/recipe-detail....   
313097           311.1  https://recipes.sparkpeople.com/recipe-detail....   
290432           248.4  https://recipes.sparkpeople.com/recipe-detail....   
480964           143.1  https://recipes.sparkpeople.com/recipe-detail....   
387524           231.3  https://recipes.sparkpeople.com/recipe-detail....   
269951           259.5  https://recipes.sparkpeople.com/recipe-detail....   
155771           824.0  https://recipes.sparkpeople.com/recipe-detail....   
551593           104.5  https://recipes.sparkpeople.com/recipe-detail....   
344399            76.5  https://recipes.sparkpeople.com/recipe-detail....   
345416           138.7  https://recipes.sparkpeople.com/recipe-detail....   
567723            93.0  https://recipes.sparkpeople.com/recipe-detail....   
92248            377.7  https://recipes.sparkpeople.com/recipe-detail....   
102022            77.6  https://recipes.sparkpeople.com/recipe-detail....   

        servings                      labels             filtered_labels  
241523       6.0                  {meatloaf}                  {meatloaf}  
260712       6.0                      {soup}                      {soup}  
444710       1.0               {green_beans}               {green_beans}  
419318      22.0          {peppermint, cups}                      {cups}  
448084       6.0                       {pie}                       {pie}  
31623       11.0                  {homemade}                  {homemade}  
20741        5.0  {stuffed_peppers, spinach}  {stuffed_peppers, spinach}  
185904       7.0      {carrots, potato_soup}      {carrots, potato_soup}  
313097       4.0                     {beans}                     {beans}  
290432       4.0                 {casserole}                 {casserole}  
480964      20.0                 {meatballs}        

In [79]:
f"Recipes without Label: ~{int(df_recipes[~label_avail_mask].shape[0] / df_recipes[label_avail_mask].shape[0] * 100)}%"

'Recipes without Label: ~19%'

### The Bad

In [92]:
bad_examples = [
    "Keema (Ground Beef Casserole) My Version",
    "My Improvised Dahl Recipe",
    "mac and cheese",
    "My version of Spaghetti Squash Baked Casserole",
    "Curry Chicken and Quinoa",
    "Eggplant Parmesan",
    "Green Diet Drink",
    "Gluten-Free Hawaiian Sweet Bread",
    "Mother's Okra & Tomato Gumbo (3/4 cup)",
    "Tortilla Soup (vegetarian)",
    "No Sugar Added Mixed Berry Crisp",
    "Amazing Tuna Salad Lettuce roll ups",
    "Southwest Bean Chili - Homemade",
]

In [148]:
df_bad_examples = df_recipes[df_recipes.title.isin(bad_examples)][
    ["title", "labels", "filtered_labels"]
]
df_bad_examples["sent"] = sents.map(lambda s: " ".join(s))
df_bad_examples = df_bad_examples.loc[
    df_bad_examples[["title"]].drop_duplicates().index
]

In [144]:
df_bad_examples

title                labels  \
187500                               Eggplant Parmesan            {parmesan}   
74626                 No Sugar Added Mixed Berry Crisp               {sugar}   
169491                                  mac and cheese          {and_cheese}   
316405                        Curry Chicken and Quinoa     {quinoa, chicken}   
487717                Gluten-Free Hawaiian Sweet Bread  {gluten_free, bread}   
375374          Mother's Okra & Tomato Gumbo (3/4 cup)                 {cup}   
243130        Keema (Ground Beef Casserole) My Version  {casserole, version}   
33533              Amazing Tuna Salad Lettuce roll ups                 {ups}   
302390                      Tortilla Soup (vegetarian)          {vegetarian}   
580420                       My Improvised Dahl Recipe              {recipe}   
259563  My version of Spaghetti Squash Baked Casserole  {casserole, version}   
31623                  Southwest Bean Chili - Homemade            {homemade}   
537263                                Green Diet Drink                {diet}   

             filtered_labels                                            sent  
187500            {parmesan}                               eggplant parmesan  
74626                {sugar}                no sugar added mixed berry crisp  
169491          {and_cheese}                                  mac and cheese  
316405     {quinoa, chicken}                        curry chicken and quinoa  
487717  {gluten_free, bread}                gluten free hawaiian sweet bread  
375374                 {cup}                    mother okra tomato gumbo cup  
243130  {casserole, version}          keema ground beef casserole my version  
33533                  {ups}             amazing tuna salad lettuce roll ups  
302390          {vegetarian}                        tortilla soup vegetarian  
580420              {recipe}                       my improvised dahl recipe  
259563  {casserole, version}  my version of spaghetti squash baked casserole  
31623             {homemade}                   southwest bean chili homemade  
537263                {diet}                                green diet drink

In [158]:
def render_spacy(title):
    sent = df_bad_examples[df_bad_examples.title == title].iloc[0].sent
    print(sent)
    doc = nlp(sent)
    return displacy.render(doc, style="dep")

In [159]:
for e in bad_examples:
    render_spacy(e)

keema ground beef casserole my version


my improvised dahl recipe


mac and cheese


my version of spaghetti squash baked casserole


curry chicken and quinoa


...

mother okra tomato gumbo cup


tortilla soup vegetarian


no sugar added mixed berry crisp


amazing tuna salad lettuce roll ups


southwest bean chili homemade
